In [10]:
valid_codes = '''AUS,AUT,BEL,BGR,HRV,CHL,CZE,DNK,GBR,EST,FIN,FRA,DEU,GRC,HUN,ISL,ISR,ITA,LVA,LTU,LUX,NLD,NZL,NOR,POL,PRT,KOR,RUS,SVN,SVK,ESP,CHE,SWE,TWN,USA'''.split(",")

In [11]:
#these are the countries we have ex_mort data for from country codes text file
#thank you Alex
print(valid_codes)

['AUS', 'AUT', 'BEL', 'BGR', 'HRV', 'CHL', 'CZE', 'DNK', 'GBR', 'EST', 'FIN', 'FRA', 'DEU', 'GRC', 'HUN', 'ISL', 'ISR', 'ITA', 'LVA', 'LTU', 'LUX', 'NLD', 'NZL', 'NOR', 'POL', 'PRT', 'KOR', 'RUS', 'SVN', 'SVK', 'ESP', 'CHE', 'SWE', 'TWN', 'USA']


In [12]:
import pandas as pd
import numpy as np
ex_mort = pd.read_csv("clean_combined.csv")
safety = pd.read_csv("covid-stringency-index.csv")
#only use countries that we have ex_mort data for
safety = safety[safety["Code"].isin(valid_codes)]
#only use weeks from the pandemic
ex_mort = ex_mort[ex_mort["years_relative"] >=0]


#convert to datetime to grab week number and year
safety["Day"] =  pd.to_datetime(safety["Day"])

In [13]:
safety["week"] = safety["Day"].dt.week
safety['year'] = safety["Day"].dt.year

/tmp/ipykernel_1733/2192045428.py:1: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  safety["week"] = safety["Day"].dt.week


In [22]:
si_df = safety.groupby(["Code", "week", "year"])["stringency_index"].mean().reset_index()
si_df = si_df.rename(columns={"Code": "country"})
si_df.head(10)

,country,week,year,stringency_index
0,AUS,1,2021,70.830000
1,AUS,1,2022,43.451429
2,AUS,2,2021,55.090000
3,AUS,2,2022,44.244286
4,AUS,3,2021,55.090000
5,AUS,3,2022,64.221429
6,AUS,4,2020,1.853333
7,AUS,4,2021,57.338571
8,AUS,4,2022,64.680000
9,AUS,5,2020,7.940000


In [15]:
#put the stringency index averages per week into the rest of the data
df = pd.merge(ex_mort, si_df)
df.head(10)

,date,country,week,year,DTotal,mov_avg,years_relative,pred_deaths,excess_mortality,em_per_capita,...,new_cases,total_deaths,new_deaths,total_cases_per_million,hosp_patients,population,mov_avg_cases,new_cases_per_capita,normalized_cases,stringency_index
0,2020-02-02,CHL,4,2020,1964.0,2006.714286,0,1875.178571,131.535714,6.846410e-06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1,2020-02-02,SWE,4,2020,1730.0,1791.000000,0,1861.609774,-70.609774,-6.949672e-06,...,1.0,NaN,0.0,0.098,NaN,10160159.0,1.0,9.842366e-08,3.057224e-06,0.0
2,2020-02-02,ESP,4,2020,9954.0,9006.428571,0,9470.538910,-464.110338,-8.516927e-05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
3,2020-02-02,DNK,4,2020,1033.0,1079.714286,0,1063.016484,16.697802,1.990192e-07,...,10.0,NaN,0.0,0.012,NaN,83900471.0,10.0,1.191888e-07,3.702230e-06,0.0
4,2020-02-02,LVA,4,2020,555.0,572.000000,0,555.143985,16.856015,1.814230e-06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
5,2020-02-02,GRC,4,2020,2774.0,2668.285714,0,2680.928571,-12.642857,-1.853598e-07,...,2.0,NaN,0.0,0.029,NaN,68207114.0,2.0,2.932245e-08,9.108106e-07,0.0
6,2020-02-02,BEL,4,2020,2204.0,2263.142857,0,2273.806015,-10.663158,-9.166826e-07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
7,2020-02-02,LUX,4,2020,86.0,84.571429,0,84.822932,-0.251504,-4.902112e-09,...,12.0,NaN,0.0,0.078,NaN,51305184.0,7.0,1.364385e-07,4.238035e-06,0.0
8,2020-02-02,KOR,4,2020,6490.0,6244.000000,0,6382.580952,-138.580952,-2.703333e-05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
9,2020-02-02,HUN,4,2020,2723.0,2580.000000,0,2673.120301,-93.120301,-8.979131e-06,...,0.0,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,0.0


In [17]:
#loading vaccination data, converting date to week/year
owidVac = pd.read_csv("vaccinations.csv")
owidVac = owidVac[owidVac["iso_code"].isin(valid_codes)]

owidVac["date"] = pd.to_datetime(owidVac["date"])
owidVac["week"] = owidVac["date"].dt.isocalendar().week
owidVac['year'] = owidVac["date"].dt.year

In [25]:
#sort vaccination data, prepare for merge
vc = owidVac.groupby(["iso_code", "week", "year", "people_vaccinated", "people_fully_vaccinated"])["total_vaccinations"].max().reset_index()
vc = vc.rename(columns={"iso_code": "country"})
vc.head(10)

,country,week,year,people_vaccinated,people_fully_vaccinated,total_vaccinations
0,AUS,1,2022,20463839.0,19782523.0,42812691.0
1,AUS,1,2022,20475452.0,19806589.0,43024593.0
2,AUS,1,2022,20488662.0,19832376.0,43286155.0
3,AUS,1,2022,20503096.0,19857649.0,43558448.0
4,AUS,1,2022,20516623.0,19882365.0,43823668.0
5,AUS,1,2022,20524868.0,19896355.0,43987111.0
6,AUS,1,2022,20529932.0,19904821.0,44086608.0
7,AUS,2,2022,20578901.0,19928352.0,44374463.0
8,AUS,2,2022,20644440.0,19946441.0,44700720.0
9,AUS,2,2022,20714403.0,19967401.0,45045755.0


In [26]:
#merge vaccination data into dataset
df = pd.merge(df, vc)
df.head(10)

,date,country,week,year,DTotal,mov_avg,years_relative,pred_deaths,excess_mortality,em_per_capita,...,total_cases_per_million,hosp_patients,population,mov_avg_cases,new_cases_per_capita,normalized_cases,stringency_index,total_vaccinations,people_vaccinated,people_fully_vaccinated
0,2020-12-20,USA,50,2020,81952.0,71991.142857,0,57378.114286,14613.028571,0.000044,...,49928.031,109689.0,332915074.0,1.246826e+06,0.003745,0.116332,71.760000,31762.0,26321.0,6449.0
1,2020-12-27,ISR,51,2020,1007.0,925.571429,0,888.566165,37.005263,0.000004,...,31671.670,7097.0,9634162.0,2.949200e+04,0.003061,0.095086,70.505714,7462.0,7458.0,4.0
2,2020-12-27,USA,51,2020,82864.0,74772.857143,0,57984.942857,16787.914286,0.000050,...,54561.498,114648.0,332915074.0,1.320415e+06,0.003966,0.123198,71.760000,1220381.0,1186226.0,11044.0
3,2021-01-03,USA,52,2020,84314.0,77169.571429,1,62503.428571,14666.142857,0.000044,...,58449.901,121424.0,332915074.0,1.395096e+06,0.004191,0.130166,71.760000,3072162.0,3000281.0,21357.0
4,2021-01-03,CHE,52,2020,2117.0,2159.000000,1,1329.344547,829.655453,0.000399,...,55479.734,1221.0,2078723.0,1.000629e+04,0.004814,0.149522,60.190000,1132.0,1107.0,1.0
5,2021-01-03,DEU,52,2020,25554.0,22585.428571,1,18255.803250,4329.625322,0.000064,...,38606.048,24645.0,67422000.0,9.633171e+04,0.001429,0.044381,82.410000,24376.0,24357.0,19.0
6,2021-01-03,EST,52,2020,360.0,333.285714,1,265.170077,68.115638,0.000001,...,40205.466,10128.0,46745211.0,6.709629e+04,0.001435,0.044585,55.560000,191.0,191.0,3.0
7,2021-01-03,ISR,52,2020,1016.0,941.428571,1,909.142831,32.285740,0.000003,...,32869.387,6261.0,9634162.0,2.671914e+04,0.002773,0.086146,74.865714,389019.0,389015.0,4.0
8,2021-01-10,CHE,1,2021,1896.0,2123.142857,1,1314.325349,808.817508,0.000389,...,60545.825,1209.0,2078723.0,1.056757e+04,0.005084,0.157909,60.190000,40509.0,39782.0,71.0
9,2021-01-10,CHE,53,2020,1982.0,2098.428571,1,NaN,NaN,NaN,...,60545.825,1209.0,2078723.0,1.056757e+04,0.005084,0.157909,60.190000,6382.0,6280.0,12.0


In [ ]:
df.plot.scatter("stringency_index", "em_per_capita")

In [ ]:
import seaborn
import matplotlib.pyplot as plt
seaborn.set(style='ticks')
seaborn.relplot(data=df, x='stringency_index', y='em_per_capita', hue='year',  aspect=1.61)
plt.show()

In [ ]:
year2 = df[df["year"] == 2021]

In [ ]:
seaborn.relplot(data=year2, x='stringency_index', y='em_per_capita', aspect=1.61)
plt.show()

In [ ]:
pd.plotting.scatter_matrix(df[["year", "em_per_capita", "normalized_cases", "stringency_index"]])
plt.show()

In [ ]:
seaborn.relplot(data=year2, x='stringency_index', y='normalized_cases', aspect=1.61)
plt.show()

In [ ]:
from sklearn import linear_model
df = df[df['normalized_cases'].notna()]
reg = linear_model.LinearRegression()
testx = df[["stringency_index"]]
testy = df["normalized_cases"]
reg.fit(testx, testy)

In [ ]:
reg.score(testx, testy)

In [ ]:
reg.intercept_

In [ ]:
y_pred = reg.predict(testx)
plt.scatter(testx, testy, color ='b')
plt.plot(testx, y_pred, color ='k')
  
plt.show()

In [ ]:
ridgereg = linear_model.Ridge(0.5)
ridgereg.fit(testx,testy)

In [ ]:
import datetime
ex_mort = pd.read_csv("clean_combined.csv")
safety = pd.read_csv("covid-stringency-index.csv")
#only use countries that we have ex_mort data for
safety = safety[safety["Code"].isin(valid_codes)]
#only use weeks from the pandemic
ex_mort = ex_mort[ex_mort["years_relative"] >=0]

for i in range(6):
    d = datetime.timedelta(days=i*7)
    #convert to datetime to grab week number and year
    safety["Day"] =  pd.to_datetime(safety["Day"])+d

    safety["week"] = safety["Day"].dt.week
    safety['year'] = safety["Day"].dt.year
    si_df = safety.groupby(["Code", "week", "year"])["stringency_index"].mean().reset_index()
    si_df = si_df.rename(columns={"Code": "country"})
    si_df.head(10)

    df = pd.merge(ex_mort, si_df)
    seaborn.relplot(data=df, x='stringency_index', y='em_per_capita', hue='year', aspect=1.61).set(title = "{} day offset".format(i*7))
    plt.show()